In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
#like_click, share
data_path = '../data'
like_click_path = os.path.join(data_path, 'like_click.csv')
# share_path = os.path.join(data_path, 'click_share_musinsa_log.csv')
# view
view_image_path = os.path.join(data_path, 'view.csv')
#outfit
outfit_path = os.path.join(data_path, 'outfit.csv')

like_click_df = pd.read_csv(like_click_path)
# share_df = pd.read_csv(share_path)
view_df = pd.read_csv(view_image_path)
outfit_df = pd.read_csv(outfit_path)

In [ ]:
#like_click데이터를 합침
like_click_df = like_click_df.groupby(['session_id', 'outfit_id'], as_index=False).agg({
    'like_type': lambda x: ', '.join(x.dropna()).strip() or np.nan,
    'click_type': lambda x: ', '.join(x.dropna()).strip() or np.nan
})

In [ ]:
like_click_df

In [ ]:
#like:1 / click:2 / both:3
like_click_df['implicit'] = np.where((~like_click_df['click_type'].isnull()) & (~like_click_df['like_type'].isnull()), 3,
                                    np.where(~like_click_df['like_type'].isnull(), 1, 2))

In [ ]:
# like_click_df[like_click_df['implicit']==1]

In [ ]:
like_click_df = like_click_df[['session_id', 'outfit_id', 'implicit']]

In [ ]:
like_click_df

In [ ]:
view_df = view_df.drop_duplicates(subset=['session_id', 'outfit_id'], keep=False)

In [ ]:
view_df

In [ ]:
concat_df = pd.concat([view_df, like_click_df])


In [ ]:
concat_df.drop_duplicates(subset=['session_id', 'outfit_id'], keep='last', inplace=True)

In [ ]:
concat_df[concat_df['implicit']==1]

In [ ]:
# !pip install scikit-learn

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Fit and transform the 'session_id' column in your DataFrame
concat_df['session_id'] = label_encoder.fit_transform(concat_df['session_id'])


In [ ]:
concat_df

In [ ]:
# Define a function to map implicit ratings to corresponding values
def map_implicit_rating(implicit_rating):
    if implicit_rating == 1:
        return 0.9
    elif implicit_rating == 2:
        return 0.6
    elif implicit_rating == 3:
        return 1.0
    else:
        return implicit_rating

# Apply the mapping function to the 'implicit rating' column and store the result in the 'rating' column
concat_df['rating'] = concat_df['implicit'].apply(map_implicit_rating)

# Drop the 'implicit rating' column if needed
concat_df.drop(columns=['implicit'], inplace=True)

In [ ]:
concat_df['rating'] = concat_df['rating'].fillna(0)

In [ ]:
concat_df = concat_df[['session_id','outfit_id','rating']]
train_path = "../data/test.csv"
concat_df.to_csv(train_path, index=False)

In [ ]:
concat_df

In [ ]:
# Group the DataFrame by 'session_id'
grouped_df = concat_df.groupby('session_id')

# Create a function to balance and extract samples from each group
def balance_and_extract_samples(group):
    non_zero_rows = group[group['rating'] != 0]
    zero_rows = group[group['rating'] == 0]

    # Calculate the target number of non-zero rows to achieve the 2:1 ratio
    target_zero_count = int(len(non_zero_rows) * 2)

    # If there are fewer non-zero rows than the target, use all of them
    if len(zero_rows) <= target_zero_count:
        zero_sample = zero_rows
    else:
        # Randomly sample 'target_non_zero_count' non-zero rows
        zero_sample = zero_rows.sample(n=target_zero_count, random_state=42)

    # Combine non-zero sample with zero rows
    balanced_sample = pd.concat([zero_sample, non_zero_rows])

    # Randomly shuffle the rows
    balanced_sample = balanced_sample.sample(frac=1, random_state=42)

    return balanced_sample

# Apply the function to each 'session_id' group and combine the results
balanced_sample_df = grouped_df.apply(balance_and_extract_samples)

# Reset the index to convert 'session_id' back to a regular column
balanced_sample_df.reset_index(drop=True, inplace=True)

# Get the last 10 entries for each 'session_id' group as the test set
test_set = balanced_sample_df.groupby('session_id').tail(3)

# Drop the test set from the balanced_sample_df to get the training set
train_set = balanced_sample_df.drop(test_set.index)

# Save the test set to 'test.csv'
test_set.to_csv('../data/train-test/test.csv', index=False)

# Save the training set to 'train.csv'
train_set.to_csv('../data/train-test/train.csv', index=False)

In [ ]:
balanced_sample_df